In [1]:
from torchvision import datasets, transforms
import torch
from net import AlexNetPlusLatent
import numpy as np

In [2]:
bits = 48
pretrained = 91.33

In [3]:
net = AlexNetPlusLatent(bits)
net.load_state_dict(torch.load('./model/91.33'))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Use device: ' + str(device))
net.to(device)
net.eval()

Use device: cuda:0


AlexNetPlusLatent(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (remain): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p

In [4]:
def load_data():
    transform_test = transforms.Compose(
        [
         transforms.Resize(227),
         transforms.ToTensor(),
         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ]
    )
    testset = datasets.CIFAR10(root='./data', train=False, download=True,
                               transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset)
    return testloader

In [6]:
def int32_aggregate_output(_loader):
    a = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(_loader):
            _int = int32_single_output(inputs)
            b = (batch_idx, _int, int(targets))
            if batch_idx%500==0:
                print('#', end='')
            a.append(b)
        return a

In [7]:
def single_output(data):
    inputs = data.to(device)
    outputs, _ = net(inputs)
    return outputs.data

In [8]:
def int32_single_output(_data):
    _tensor = (single_output(_data)[0,:]>0.5).int()
    _str = ''
    for i in range(0, _tensor.shape[0]):
        _str += str(int(_tensor[i]))
    _int = int(_str, 2)
    return _int

In [9]:
_loader = load_data()
a = int32_aggregate_output(_loader)

Files already downloaded and verified
####################

In [37]:
a[6569][1]

31598448389273

In [50]:
def hammingDistance(x, y):
    ans = 0
    for i in range(31,-1,-1):
        b1= x>>i&1
        b2 = y>>i&1
        ans+= not(b1==b2)
    return ans

In [51]:
def test(pic_id, hash_list):
    _a = []
    _pic_hash = hash_list[pic_id][1]
    for i in range(0, len(hash_list)):
        _h_distance = hammingDistance(_pic_hash, hash_list[i][1])
        b = (hash_list[i][0], _h_distance, hash_list[i][2])
        _a.append(b)
    return _a

In [67]:
c = test(345 , a)

In [68]:
# take second element for sort
def takeSecond(elem):
    return elem[1]

# sort list with key
c.sort(key=takeSecond)

# print list
c

[(12, 0, 5),
 (345, 0, 5),
 (550, 0, 5),
 (652, 0, 5),
 (937, 0, 5),
 (1328, 0, 5),
 (1388, 0, 5),
 (1773, 0, 5),
 (2026, 0, 5),
 (2427, 0, 5),
 (2731, 0, 5),
 (2750, 0, 5),
 (2970, 0, 5),
 (3434, 0, 5),
 (3774, 0, 5),
 (3949, 0, 5),
 (3954, 0, 5),
 (3981, 0, 5),
 (4017, 0, 5),
 (4160, 0, 5),
 (4626, 0, 5),
 (5035, 0, 5),
 (5650, 0, 5),
 (5849, 0, 5),
 (5979, 0, 5),
 (6037, 0, 5),
 (6226, 0, 5),
 (7386, 0, 5),
 (7393, 0, 5),
 (7549, 0, 3),
 (7834, 0, 5),
 (8478, 0, 5),
 (9058, 0, 5),
 (9257, 0, 5),
 (9269, 0, 5),
 (9582, 0, 5),
 (9622, 0, 5),
 (9741, 0, 5),
 (16, 1, 5),
 (31, 1, 5),
 (39, 1, 5),
 (42, 1, 5),
 (61, 1, 3),
 (101, 1, 5),
 (141, 1, 5),
 (155, 1, 5),
 (168, 1, 5),
 (181, 1, 5),
 (190, 1, 5),
 (207, 1, 5),
 (212, 1, 5),
 (230, 1, 5),
 (238, 1, 5),
 (239, 1, 5),
 (250, 1, 5),
 (318, 1, 5),
 (319, 1, 5),
 (327, 1, 5),
 (361, 1, 5),
 (389, 1, 5),
 (421, 1, 5),
 (431, 1, 5),
 (444, 1, 5),
 (457, 1, 5),
 (460, 1, 5),
 (484, 1, 5),
 (502, 1, 5),
 (510, 1, 5),
 (514, 1, 5),
 (525, 